graph db
conjuctive regaulr path queires poptimization


https://arxiv.org/pdf/2407.04823 Path-based Algebraic Foundations of Graph Query Languages
https://openproceedings.org/2025/conf/edbt/paper-234.pdf Path-based Algebraic Foundations of Graph Query Languages

https://dl.acm.org/doi/abs/10.1145/3725237 Minimizing Conjunctive Regular Path Queries

https://en.wikipedia.org/wiki/Regular_path_query
https://homepages.inf.ed.ac.uk/libkin/papers/icdt12b.pdf

networkx?
sqlite?

SQL/PCQ
GQL https://en.wikipedia.org/wiki/Graph_Query_Language

RPQ regular expressions as queries

I have wanted recursive queries for embedding egraphs in graph rewriting. share*  and E*

datalog could do it by materializing the needed transitive closure?
https://docs.google.com/document/d/1eq7jED7XMnOuq80PggkVKtqWTL1wQ8SdZMIWHSZLoPM/edit?tab=t.0

filtyer push down is smaller
hjow to use cost model

https://rustmagazine.org/issue-2/write-a-sql-optimizer-using-egg/

basic graph patterns

sqlite = bad?
It does suck encoding into it

paths asd first class

GPC - graph pattern calculus

cq conjunctive query
rpq - recursive path query
two way rpq

https://github.com/egraphs-good/egg/discussions/189


In [ ]:
type Rel

class Join():
    A : Rel
    B : Rel
    cols : list[str]

class Union():
    A : Rel
    B : Rel

class Star():
    R : Rel

class Proj(): # Select, Sigma
    R : Rel
    cond : ?


In [ ]:
class Proj():
    cols : set[str]
    R : Rel
class Join():
    A : Rel
    B : Rel
    on : list[tuple[str, str]] # (A.col, B.col)
class Filter():
    vs : dict[str, any] # col -> value
    R : Rel



In [ ]:
@dataclass
class App():
    f : str
    args : tuple['App', ...] = ()
proj()

def rw():
    ...


In [ ]:

class GraphDB():
    vertices : set[object]
    edges: 


 rdflib https://rdflib.readthedocs.io/en/stable/gettingstarted/#a-sparql-query-example

In [ ]:
%%file /tmp/rel.egg

(datatype Query
    (Join Query Query) ; (Vec String)?
    (Union Query Query)
    (Star Query)
    ()
)



In [ ]:
import egglog
E = egglog.EGraph()

class Math(egglog.Expr):



In [3]:
dir(egglog)

['Action',
 'BackOff',
 'BaseExpr',
 'BigInt',
 'BigIntLike',
 'BigRat',
 'BigRatLike',
 'Bool',
 'BoolLike',
 'BuiltinExpr',
 'Command',
 'Container',
 'ConvertError',
 'CostModel',
 'EGraph',
 'EggSmolError',
 'Expr',
 'ExprCallable',
 'ExprValueError',
 'Fact',
 'GraphvizKwargs',
 'GreedyDagCost',
 'Map',
 'MapLike',
 'MultiSet',
 'Primitive',
 'PyObject',
 'Rational',
 'RewriteOrRule',
 'Ruleset',
 'Schedule',
 'Set',
 'SetLike',
 'StageInfo',
 'String',
 'StringLike',
 'TimeOnly',
 'Unit',
 'UnstableFn',
 'Vec',
 'VecLike',
 'WithPlan',
 '_BirewriteBuilder',
 '_EqBuilder',
 '_NeBuilder',
 '_RewriteBuilder',
 '_SetBuilder',
 '_UnionBuilder',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'back_off',
 'bindings',
 'birewrite',
 'builtins',
 'check',
 'check_eq',
 'config',
 'constant',
 'conversion',
 'convert',
 'converter',
 'declarations',
 'deconstruct',
 'default_cost_model',
 'define_expr_method',